In [ ]:
#Data for Mooring D

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import gsw

def calculate_buoy_freq(sals, temps, depths):
    lat = np.repeat(59.153,np.shape(sals)[0])
    lon = np.repeat(-58.895,np.shape(sals)[0])
    abs_sal = gsw.SA_from_SP(sals,depths,lat,lon)
    pot_temp = gsw.pt0_from_t(abs_sal, temps, depths)
    cons_temp = gsw.conversions.CT_from_pt(abs_sal,pot_temp)
    density = gsw.density.sigma0(abs_sal,cons_temp)
    buoy_freq = gsw.Nsquared(abs_sal, cons_temp, depths, lat)
    return buoy_freq[0]

numb_years = 17
letters = ['a', 'b', 'd']
years = ['0506', '0607', '0708', '0809', '0910', '1011', '1112', '1213', '1314', '1314', '1415', '1516', '1617', '1718', '1821','2122','2122']
files = [336, 309, 314, 377, 342, 272, 257, 307, 321, 1044, 338, 326, 308, 310, 967, 701, 701] #prob last one - 9, 14


number_file = np.sum(files[0:numb_years])
dts = np.empty([971*number_file], dtype='U25')
depths = np.empty([971*number_file])
sals = np.empty([971*number_file])
temps = np.empty([971*number_file])
us = np.empty([971*number_file])
vs = np.empty([971*number_file])
ws = np.empty([971*number_file])
buoys = np.empty([number_file])
u_maxs = np.empty([number_file])
u_max_uppers = np.empty([number_file])
dt_buoy = np.empty([number_file], dtype='U25')
dt_depth = np.empty([number_file])
shear_u = np.empty([971*number_file])

index = 0
index2 = 0

for j in range(2,3):
    #Start with the first file
    path = '/Users/helena/Downloads/bg' + years[0] + '_mmp_' + letters[j] + '/bg' + years[0] + '_mmp_' + letters[j] + '001.dat'
    data = np.loadtxt(path, dtype = str, skiprows = 2)
    header = np.loadtxt(path, dtype = str, max_rows=1)
        
    for k in range(0,numb_years): #different years files
        print(k)
        #File names
        profile_names = np.zeros((files[k],1), dtype=object)
        for i in range(1, files[k]+1):
            if k==1 and j==1:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '{0:0>4}'.format(i) + '.dat'
            elif k==14 and j==1:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==9 and j==2:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==14 and j==2:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==16 and j==2:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            else:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '{0:0>3}'.format(i) + '.dat'
            profile_names[i-1] = name
        
        #Then the next ones
        for i in range(0, files[k]):
            try:
                if profile_names[i][0] == 'bg0910_mmp_b163.dat':
                    raise IOError
                if profile_names[i][0] == 'bg0910_mmp_b171.dat':
                    raise IOError
                if profile_names[i][0] == 'bg0910_mmp_b187.dat':
                    raise IOError
                
                if k==14 and j==1:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==9 and j==2:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==14 and j==2:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==16 and j==2:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                else:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '/' + profile_names[i][0]
                
                
                header = np.loadtxt(path, dtype = str, max_rows=1)
                data = np.loadtxt(path, dtype = str, skiprows = 2)
                
                date = header[6] + ' ' + data[0][0]
                
#                 print(index)
                
                if k==9 or k==14 or k==15 or k==16:
                    mask = [True if float(i)%2!=0 else False for i in data[:,1]]
                    dt = np.repeat(date,np.shape(data[mask])[0])
                    numm = np.shape(data[mask])[0]

                    dts[index:index+numm] = dt
                    depths[index:index+numm] = data[:,1][mask]
                    sals[index:index+numm] = data[:,3][mask]
                    temps[index:index+numm] = data[:,2][mask]
                    us[index:index+numm] = data[:,4][mask]
                    vs[index:index+numm] = data[:,5][mask]
                    ws[index:index+numm] = data[:,6][mask]
                    
                    new_row = np.empty(np.shape(data[0]))
                    new_row[:] = np.nan
                    shifted = np.vstack([data[mask],new_row])
                    
                    shear_u[index:index+numm] = (data[:,4][mask].astype(float)-shifted[:,4][1:].astype(float))/2
                    
#                     buoy_freq_temp = calculate_buoy_freq(data[:,3][mask],data[:,2][mask],data[:,1][mask])
#                     masky = buoy_freq_temp<.0002
                    masky = data[:,1][mask].astype(float)<101
                    buoy_freq_temp = calculate_buoy_freq(data[:,3][mask][masky],data[:,2][mask][masky],data[:,1][mask][masky])
        
#                     if buoy_freq_temp[masky].size:
#                         buoys[index2] = np.nanmax(buoy_freq_temp[masky])
                    if buoy_freq_temp.size and np.isnan(buoy_freq_temp).all()==False:
                        buoys[index2] = np.nanmax(buoy_freq_temp)
                        ind = np.nanargmax(buoy_freq_temp)
                        dt_depth[index2] = data[:,1][mask][masky][ind+1]
                        dt_buoy[index2] = date
                
                    if us[index:index+numm].size:
                        u_maxs[index2] = np.nanmax(us)
#                         u_max_uppers[index2] = np.nanmax(data[data[:,1]<'101'][:,4].astype(float))
                        

                    index = np.shape(data[mask])[0] + index
                    index2 = index2+1
                    
                else:
                    dt = np.repeat(date,np.shape(data)[0])
                    numm = np.shape(data)[0]

                    dts[index:index+numm] = dt
                    depths[index:index+numm] = data[:,1]
                    sals[index:index+numm] = data[:,3]
                    temps[index:index+numm] = data[:,2]
                    us[index:index+numm] = data[:,4]
                    vs[index:index+numm] = data[:,5]
                    ws[index:index+numm] = data[:,6]
                    
                    new_row = np.empty(np.shape(data[0]))
                    new_row[:] = np.nan
                    shifted = np.vstack([data,new_row])
                    shear_u[index:index+numm] = (data[:,4].astype(float)-shifted[:,4][1:].astype(float))/2
                    
#                     buoy_freq_temp = calculate_buoy_freq(data[:,3],data[:,2],data[:,1])
#                     masky = buoy_freq_temp<.0002
                    masky = data[:,1].astype(float)<101
                    buoy_freq_temp = calculate_buoy_freq(data[:,3][masky],data[:,2][masky],data[:,1][masky])

#                     if buoy_freq_temp[masky].size:
#                         buoys[index2] = np.nanmax(buoy_freq_temp[masky])
                    if buoy_freq_temp.size and np.isnan(buoy_freq_temp).all()==False:
                        buoys[index2] = np.nanmax(buoy_freq_temp)
                        ind = np.nanargmax(buoy_freq_temp)
                        dt_depth[index2] = data[:,1][masky][ind+1]
                        dt_buoy[index2] = date
#                         print(np.shape(buoy_freq_temp))
#                         print(ind)
#                         print(np.shape(data[:,1][masky]))
#                         print(np.shape(data[:,3][masky]))
#                         print(np.shape(data[:,2][masky]))
#                         print(np.shape(data[:,1][masky]))
                        
                    if us[index:index+numm].size:
                        u_maxs[index2] = np.nanmax(us)
#                         u_max_uppers[index2] = np.nanmax(data[data[:,1]<'101'][:,4].astype(float))

                    index = np.shape(data)[0] + index
                    index2 = index2 + 1
                
#                 fig = go.Figure(data=go.Heatmap(
#                     z=data[:,3],
#                     x=dt,
#                     y=data[:,1],
#                     colorscale='Viridis'))

            except IOError:
                0
#                 print('The data for profile number ' + profile_names[i][0] + ' does not exist')
            except IndentationError:
                print("hopefully nothing happens")
    if j==0:
        data_a = data
    if j==1:
        data_b = data
    if j==2:
        data_d = data



0
1
2
3
4
5
6
7
8
9


In [ ]:
#Salinity Data

fig = go.Figure(data=go.Heatmap(
                    z=sals,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'psu',zmin = 30, zmax = 35))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Salinity - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/salinity_d.html")

In [ ]:
#Temperature Data

fig = go.Figure(data=go.Heatmap(
                    z=temps,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = '°C'))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Temperature - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/temperature_d.html")

In [ ]:
#Speed Data

speeds = np.sqrt(np.square(us) + np.square(vs))
fig = go.Figure(data=go.Heatmap(
                    z=speeds,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'cm/s'))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Speed - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/speeds_d.html")

In [ ]:
#Buoyancy Frequency Data

lat = np.repeat(59.153,np.shape(sals)[0])
lon = np.repeat(-58.895,np.shape(sals)[0])
masky = buoy_freq[0]<.0002

abs_sal = gsw.SA_from_SP(sals,depths,lat,lon)
pot_temp = gsw.pt0_from_t(abs_sal, temps, depths)
cons_temp = gsw.conversions.CT_from_pt(abs_sal,pot_temp)
density = gsw.density.sigma0(abs_sal,cons_temp)
buoy_freq = gsw.Nsquared(abs_sal, cons_temp, depths, lat)

fig = go.Figure(data=go.Heatmap(
                    z=buoy_freq[0][masky],
                    x=dts[1:][masky],
                    y=depths[1:][masky],
                    colorscale='Viridis',
                    colorbar_title = '1/s', zmin = 0, zmax = .0006))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Buoyancy Frequency - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/buoy_freq_d.html")

In [ ]:
#Buoyancy Frequency Data with Max Buoyancy Frequency Line

heatmap = go.Heatmap(z=buoy_freq[0],x=dts[1:],y=depths[1:],colorscale='Viridis',colorbar_title = '1/s', zmin = 0, zmax = .0006)
lines = go.Scatter(x=dt_buoy, y = dt_depth, mode='lines', line_color='black', line_width=1)

fig = go.Figure(data=[heatmap,lines])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Buoyancy Frequency and Max Line - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/buoy_freq_line_d.html")

In [ ]:
#Buoyancy Frequency For Eddy Analysis

heatmap = go.Heatmap(z=buoy_freq[0],x=dts[1:],y=depths[1:],colorscale='Viridis', colorbar_title = '1/s', zmin = 0, zmax = .0006)
fig = go.Figure(data=[heatmap])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Test_BF_D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/test_bf_d.html")

/Users/helena/anaconda3/lib/python3.8/site-packages/gsw/stability.py:86: RuntimeWarning: divide by zero encountered in true_divide
  N2 = ((g_local**2) / (specvol_mid * db_to_pa * dp))
/Users/helena/anaconda3/lib/python3.8/site-packages/gsw/stability.py:87: RuntimeWarning: invalid value encountered in multiply
  N2 *= (beta_mid*dSA - alpha_mid*dCT)


In [ ]:
#Vertical Shear for Eddy Analysis

heatmap = go.Heatmap(z=shear_u,x=dts[1:],y=depths[1:], colorscale='Viridis', colorbar_title = '1/s', zmin=-2, zmax=2, zsmooth = 'best')
fig = go.Figure(data=[heatmap])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Test_SH_D_Smooth")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_image("/Users/helena/Downloads/test_shear_d_smooth.png")

In [ ]:
#Richardson Number for Eddy Analysis

heatmap = go.Heatmap(z=np.abs(buoy_freq[0]/shear_u[:-1]),x=dts[1:],y=depths[1:],colorscale='Viridis', zmin=0,zmax=.0004)
fig = go.Figure(data=[heatmap])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Test_R_abs_D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/test_R_d.html")

In [92]:
#Absolute Value of Richardson Number for Eddy Analysis

heatmap = go.Heatmap(z=np.abs(buoy_freq[0]/shear_u[:-1]),x=dts[1:],y=depths[1:],colorscale='Viridis', zmin=0,zmax=.0004)
fig = go.Figure(data=[heatmap])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Test_R_abs_D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/test_R_abs_d.html")

<ipython-input-92-2c53fcc797fd>:1: RuntimeWarning:

divide by zero encountered in true_divide



In [93]:
#U velocity for Eddy Analysis

heatmap = go.Heatmap(z=us,x=dts[1:],y=depths[1:],colorscale='Viridis',colorbar_title = 'cm/s',)
fig = go.Figure(data=[heatmap])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Test_U_D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/test_us_d.html")

In [1]:
#Visualization of Thermohaline Staircase in the Arctic

import matplotlib.pyplot as plt
n = 0
plt.plot(temps[n*971:970*(n+1)],depths[n*971:970*(n+1)])
plt.ylim(600,200)
plt.title('Arctic Ocean Temperature Staircase - Mooring D')
plt.xlabel('Temperature')
plt.ylabel('Depth [m]')
plt.show()

NameError: name 'temps' is not defined

In [3]:
#U Velocity Plot

fig = go.Figure(data=go.Heatmap(
                    z=us,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'cm/s'))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="U velocity - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/us_d.html")

In [4]:
#V Velocity Plot
fig = go.Figure(data=go.Heatmap(
                    z=vs,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'cm/s'))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="V velocity - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/vs_d.html")

In [7]:
#W Velocity Plot
fig = go.Figure(data=go.Heatmap(
                    z=np.abs(ws),
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'cm/s'))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="W velocity - Mooring D")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/ws_d.html")